In [1]:

import pandas as pd
import urllib.request
import json


In [2]:
# URLs de los archivos Excel
export_url = 'https://www.dian.gov.co/dian/cifras/PrincipalesproductosimpexpDS/Principales-productos-exportados-DS.xlsx'
import_url = 'https://www.dian.gov.co/dian/cifras/PrincipalesproductosimpexpDS/Principales-productos-importados-DS.xlsx'

iterable = [export_url, import_url]


In [9]:
# Función para descargar y convertir en DataFrame
def download_file(url):
    # Realizar la solicitud y obtener el contenido del archivo
    local_filename = path + url.split('/')[-1]

    try:
        urllib.request.urlretrieve(url, local_filename)
        return f'ok'
    except HTTPError as e:
        return f'Error HTTP al descargar el archivo: {e.code}'
    except URLError as e:
        return f'Error de URL al descargar el archivo: {e.reason}'
    except Exception as e:
        return f'Error desconocido: {e}'
    

def convert2df(iterable):

    excelfile = path + iterable.split('/')[-1]

    xl = pd.ExcelFile(excelfile)

    # Itera sobre cada hoja en el workbook
    df_total = pd.DataFrame()

    for sheet_name in xl.sheet_names:
        print(sheet_name)
        if sheet_name == 'Actualización':
            df = xl.parse(sheet_name)
            data = df.columns[df.any()]

            info_dataframe = ''.join([f'{x} \n' for x in df[data[0]] if not pd.isna(x)])
            print(info_dataframe)
        else:
            df = xl.parse(sheet_name)
            df = df.iloc[:, 1:]
            
            while True:
                df.columns = df.iloc[0, :]
                df = df.iloc[1:, :]
                df.reset_index(inplace=True)
                if 'Año' in df.columns:
                    break

            df = df.dropna(how='all')
            vals = []
            for x in df['Año']:
                if not pd.isna(x):
                    val = x
                vals.append(val)
            df['Año'] = vals
            df['SeccionalAduana'] = sheet_name
            try:
                df = df.dropna(subset="Producto")
                df = df.drop(df[df['Producto'].str.contains('Total')].index)
            except:
                pass

            df_total = pd.concat([df_total, df], axis=0, ignore_index=True)
    df_total = df_total[['SeccionalAduana', 'Año', 'Producto', 'Valor FOB US$', ' Peso Neto en Kilogramos']]
    info_dataframe += '\n Caracteristicas: '+', '.join(df_total.columns)+'\nTamaño en filas: '+str(df_total.shape[0])
    return {'info_source': info_dataframe, 'data_json': df_total.to_dict(orient='list')}

In [10]:
path = 'datos/'
list(map(download_file, iterable))
import_export = list(map(convert2df, iterable))

Actualización
UAE DIAN  - SUBDIRECCIÓN DE  ESTUDIOS ECONÓMICOS - DIRECCIÓN DE GESTIÓN ESTRATÉGICA Y DE ANALÍTICA 
PRINCIPALES PRODUCTOS EXPORTADOS POR DIRECCIÓN SECCIONAL 
2018-2023 
Fecha actualización 28-11-2023 
Producción DIAN y Certificación DANE septiembre de 2023 

Armenia
Bogotá
Bucaramanga
Pereira
Santa Marta
Riohacha
San Andrés
Arauca
Buenaventura
Ipiales
Leticia
Maicao
Tumaco
Urabá
Puerto Asís
Cartagena
Barranquilla
Cali
Cúcuta
Medellín
Actualización
UAE DIAN  - SUBDIRECCIÓN DE ESTUDIOS ECONÓMICOS - DIRECCIÓN DE GESTIÓN ESTRATÉGICA Y DE ANALÍTICA 
PRINCIPALES PRODUCTOS IMPORTADOS POR DIRECCIÓN SECCIONAL 
2018-2023* 
Fecha actualización 28-11-2023 
Producción DIAN y Certificación DANE a septiembre de 2023 

Armenia
Bogotá
Bucaramanga
Manizales
Palmira
Pereira
Popayán
Santa Marta
Tuluá
Tunja
Villavicencio
Valledupar
Riohacha
San Andrés
Arauca
Buenaventura
Ipiales
Leticia
Maicao
Tumaco
Urabá
Yopal
Puerto Asís
Cartagena
Barranquilla
Cali
Cúcuta
Medellín


In [ ]:
with open(path + "import_export.json", "w") as archivo:
    json.dump(import_export, archivo, indent=4, ensure_ascii=False)

### La siguiente celda es solo de caracter informativo de contenido de datos

In [15]:
data = import_export[0]
print(data.keys())
print(data['info_source'])

df = pd.DataFrame.from_dict(data['data_json'])
df.head()

dict_keys(['info_source', 'data_json'])
UAE DIAN  - SUBDIRECCIÓN DE  ESTUDIOS ECONÓMICOS - DIRECCIÓN DE GESTIÓN ESTRATÉGICA Y DE ANALÍTICA 
PRINCIPALES PRODUCTOS EXPORTADOS POR DIRECCIÓN SECCIONAL 
2018-2023 
Fecha actualización 28-11-2023 
Producción DIAN y Certificación DANE septiembre de 2023 

 Caracteristicas: SeccionalAduana, Año, Producto, Valor FOB US$,  Peso Neto en Kilogramos
Tamaño en filas: 1195


,SeccionalAduana,Año,Producto,Valor FOB US$,Peso Neto en Kilogramos
0,Armenia,2018,Carburorreactores tipo queroseno para reactore...,1192358.47,1551278.51
1,Armenia,2019,Carburorreactores tipo queroseno para reactore...,1060166.38,1460957.70
2,Armenia,2019,Alcohol etílico sin desnaturalizar con grado a...,7040.10,1200.00
3,Armenia,2020,Carburorreactores tipo queroseno para reactore...,1927864.91,2729424.06
4,Armenia,2021,Carburorreactores tipo queroseno para reactore...,843905.37,1078047.94
